In [2]:
import os
from typing import List

import faiss, chromadb
from llama_index.readers.file import PDFReader
from llama_index.core.schema import Document, TextNode
from llama_index.core.node_parser import SemanticSplitterNodeParser
from langchain_community.embeddings import OllamaEmbeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
from langchain_community.vectorstores import FAISS
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex


c:\Users\SARAN KAANTH L\AppData\Local\pypoetry\Cache\virtualenvs\ai_agent_based_hospital_chatbot-QmuN_Jek-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading PDF File

In [3]:
file_path="..\\data\\nabh_gib_hos.pdf"

pdf_reader=PDFReader()
documents:List[Document]=pdf_reader.load_data(file_path)

In [ ]:
documents

### Semantic Chunking

In [4]:
# embedding_model=OpenAIEmbedding()
# embedding_model=OllamaEmbeddings(model_name="nomic-embed-text",show_progress=True)
embedding_model=HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")
semantic_splitter=SemanticSplitterNodeParser(buffer_size=1,embed_model=embedding_model,breakpoint_percentile_threshold=95)

c:\Users\SARAN KAANTH L\AppData\Local\pypoetry\Cache\virtualenvs\ai_agent_based_hospital_chatbot-QmuN_Jek-py3.11\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SARAN KAANTH L\AppData\Local\llama_index\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
nodes:List[TextNode]=semantic_splitter.get_nodes_from_documents(documents,show_progress=True)

Parsing nodes: 100%|██████████| 21/21 [00:04<00:00,  4.42it/s]


### Creating Embeddings and Indexing using FAISS

In [ ]:
dim=1536
faiss_index=faiss.IndexFlatL2()

In [ ]:
vector_store=FaissVectorStore(faiss_index)
storage_context=StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
vector_index=VectorStoreIndex(nodes=nodes, embed_model=embedding_model, storage_context=storage_context)
vector_index.storage_context.persist("../storage")

In [6]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("nabh_data")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context,embed_model=embedding_model)

### Retrieval

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings

In [ ]:
vector_store=FaissVectorStore.from_persist_dir("../storage")
storage_context=StorageContext.from_defaults(vector_store=vector_store,persist_dir="../storage")
vector_index=load_index_from_storage(storage_context)

In [ ]:
retriever=vector_index.as_retriever(kwargs={"similarity_top_k":3})

In [ ]:
docs=retriever.retrieve(str_or_query_bundle='what are the certification programs provided by nabh')

In [ ]:
docs[0].node.text

In [ ]:
docs

In [ ]:
embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("nabh_collection")

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
documents

In [ ]:
index=VectorStoreIndex.from_documents(documents, storage_context=storage_context, embed_model=embed_model)

In [ ]:
db = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text",show_progress=True))

## Retriever using Langchain and FAISS with HuggingFace

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

In [3]:
file_path="..\\data\\nabh_gib_hos.pdf"

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [4]:
len(pages)

21

In [16]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1250,chunk_overlap=100)

In [17]:
split_docs=text_splitter.split_documents(pages)

In [18]:
len(split_docs)

34

In [19]:
split_docs[10]

Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20250227212411', 'source': '..\\data\\nabh_gib_hos.pdf', 'total_pages': 21, 'page': 6, 'page_label': '7'}, page_content='NATIONAL ACCREDITATION BOARD FOR HOSPITALS  \n& HEALTHCARE PROVIDERS (NABH) \n   \nPage 7  \nBenefits of Accreditation \n \n \n  \n \n \nBenefits for Staff \n \n• The staff in an accredited Hospital is satisfied lot as it provides for continuous \nlearning, good working environment and leadership.  \n• Efficiencies and competencies of staff also gets improved in an accredited Hospital.  \n• It improves overall professional de velopment, knowledge and competencies in \nsystematic ways with defined ownership and accountability of all the staff including \nMedical and Para Medical Staff.  \n \nBenefits to paying and regulatory bodies \n \nFinally, accreditation provides an objective system of  empanelment by insurance and \nother third parties. Accreditation provides access to reliable and 

In [20]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\SARAN KAANTH L\AppData\Local\Temp\ipykernel_22636\3123569051.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\SARAN KAANTH L\AppData\Local\pypoetry\Cache\virtualenvs\ai_agent_based_hospital_chatbot-QmuN_Jek-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SARAN KAANTH L\AppData\Local\pypoetry\Cache\virtualenvs\ai_agent_based_hospital_chatbot-QmuN_Jek-py3.11\Lib\

In [21]:
vector_store=FAISS.from_documents(split_docs,embeddings)

In [22]:
vector_store_name="faiss_storage"
vector_store.save_local(vector_store_name)

In [23]:
result=vector_store.similarity_search(query="What are the certification programs provided by nabh?",k=5)

In [ ]:
result[4].page_content

'NATIONAL ACCREDITATION BOARD FOR HOSPITALS  \n& HEALTHCARE PROVIDERS (NABH) \n   \nPage 11  \nAccreditation Process'